In [ ]:
#| label: setup
#| include: false
import markdown
import pins
from pins import board_connect
import os
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
import plotly.express as px
import plotly.io as pio
import altair as alt
from great_tables import GT, md
from IPython.display import display, Markdown, Latex

connect_server = "https://colorado.posit.co/rsc"
connect_api_key = os.environ.get("CONNECT_API_KEY")

board = board_connect(server_url=connect_server, api_key=connect_api_key)
df = board.pin_read("isabella.velasquez/bls-cpi-data")

pio.templates.default = "plotly_white"

# Value for last updated date
df["Year-Month"] = pd.to_datetime(df["Year-Month"])
last_updated_date = df["Year-Month"].max().strftime("%B  %Y")

# Value for the percentage change from the previous month
cpi_df = df[df["Category"] == "CUUR0000SA0"] # All groups
latest_cpi_value = cpi_df["value"].iloc[-1]
latest_cpi_percent_change = cpi_df["Percent_Change_from_Previous_Month"].iloc[-1]

# Reformatted data for display table
pivot_df = df.pivot_table(
    index="Category_Label",
    columns="Year-Month",
    values="Percent_Change_from_Previous_Month",
    aggfunc="first",
)
pivot_df = pivot_df.iloc[:, -5:].iloc[:, ::-1]
pivot_df.columns = [pd.to_datetime(col).strftime('%B %Y') for col in pivot_df.columns]
pivot_df.reset_index(inplace=True)
pivot_df.columns.name = None

## Row {height=20%}


In [ ]:
#| label: valuebox1
last_updated_date

In [ ]:
#| label: valuebox2
dict(
  value=latest_cpi_value.round(2)
  )

In [ ]:
#| label: valuebox3
dict(
  value = latest_cpi_percent_change.round(2)
  )

## Row {height=80%}

### Column {width=20%}

The **Consumer Price Index (CPI)** is a measure of the average change over time in the prices paid by urban consumers for a market basket of consumer goods and services. 

Indexes are available for the U.S. and various geographic areas. Average price data for select utility, automotive fuel, and food items are also available.

Source: 

Bureau of Labor Statistics

### Column {.tabset width=80%}


In [ ]:
#| label: pct-change-jan-2018-code
#| include: false
df = df.sort_values(by=["Category_Label", "Year-Month"], ascending=[False, False])

fig = px.line(
    df,
    x="Year-Month",
    y="Percent_Change_from_January_2018",
    color="Category_Label",
    hover_name="Category_Label",
)

for d in fig["data"]:
    if d["name"] == "All groups CPI":
        d["line"]["color"] = "orange"
        d["legendgroup"] = 1
    else:
        d["line"]["color"] = "lightgrey"

fig.update_xaxes(title_text="")
fig.update_yaxes(title_text="")

In [ ]:
#| label: pct-change-jan-2018-plot
display(
    Markdown(
        "Use this inflation data explorer to see which goods and services are getting cheaper or more expensive in the U.S."
    )
)
fig.show()

In [ ]:
#| label: pct-change-monthly-code
#| include: false
latest_month_data = df[df["Year-Month"] == df["Year-Month"].max()]

latest_month_data_sorted = latest_month_data.sort_values(
    by="Percent_Change_from_Previous_Month", ascending=False
)

top_six_categories = latest_month_data_sorted.head(6)

fig = px.bar(
    top_six_categories,
    x="Category_Label",
    y="Percent_Change_from_Previous_Month",
    title="",
    color="Category_Label",
    color_discrete_sequence=px.colors.sequential.Reds[::-1],
)

fig.update_layout(
    xaxis_title="", yaxis_title="Percent Change", yaxis_tickformat="%.1f%%"
)

In [ ]:
#| label: pct-change-monthly-plot
display(
    Markdown(
        "Percentage change in CPI by category for the latest month, top six categories"
    )
)
fig.show()

In [ ]:
#| label: pct-change-monthly-table
(
    GT(pivot_df)
    .tab_header(
        title="Monthly changes in prices for various categories",
        subtitle=md("**Not seasonally adjusted**"),
    )
    .cols_label(Category_Label="")
    .data_color(
        domain=[-10, 10],
        palette=["orange", "white", "red"],
        na_color="white",
    )
    .tab_spanner(
        label="Time",
        columns=~cs.first()
    )
    .tab_source_note(source_note="Source: Bureau of Labor Statistics")
)